# Below is how I approach the Movie project
### 1. Frame the Problem and Look at the Big Picture
1. Define the objective in business terms.
    - There are two objectives namely knowing the projected revenue and customer interests.
2. How will your solution be used?
    - Solution will be used by companies to understand what profits to expect based on various factors.
    - Movie lovers can get recommendation of movies based on movies they've seen.
3. What are the current solutions/workarounds (if any)?
    - Currently no solutions as I'm building from scratch.
4. How should you frame this problem (supervised/unsupervised,online/offline, etc.)?
    - Supervised and online.
5. How should performance be measured?
    - For revenue will use rsme.
    - For recommender system will figure out a performance measure.
6. Is the performance measure aligned with the business objective?
    - Yes.
7. What would be the minimum performance needed to reach the business objective?
    - For revenue a score of 94% is acceptable. 
    - For recommender system will figure out but something similar to revenue score. 
8. What are comparable problems? Can you reuse experience or tools?
    - Assume none exist.
9. Is human expertise available?
    - Assume no.
10. How would you solve the problem manually?
    - For revenue talking to production managers and people in the movie industry.
    - For recommendor asking random people what other movies they love after identifying a base of movies they love.
11. List the assumptions you (or others) have made so far.
    - None identified at this point. 
12. Verify assumptions if possible.
    - None to verify. 

In [ ]:
# Deleted all the previous code I had since I realized that using data from someone else makes the work harder.
# Other reason is that I can easily get the same data plus I noted some inconsistencies with the previous data during EDA. 